In [18]:
# Imports
import numpy as np
import pandas as pd
import joblib
import time
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
from sklearn.metrics import classification_report
from scipy.sparse import lil_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score

start_time = time.time()

### Isolation Forest with Sequence Data
This script trains an Isolation Forest model. It uses the event sequences from HDFS.npz and converts them into a numerical feature matrix (event counts) before training.

In [19]:
# File Paths
def locate_project_root(marker_path: str = "data/HDFS.npz", max_depth: int = 5) -> Path:
    current = Path.cwd()
    for _ in range(max_depth + 1):
        if (current / marker_path).exists():
            return current
        current = current.parent
    raise FileNotFoundError(f"Unable to locate '{marker_path}' within {max_depth} parent levels from {Path.cwd()}.")

PROJECT_ROOT = locate_project_root()
DATA_PATH = PROJECT_ROOT / "data" / "HDFS.npz"
MODEL_OUTPUT_PATH = PROJECT_ROOT / "ml_models" / "isolation_forest_model.joblib"

print(f"Using project root: {PROJECT_ROOT}")
print(f"Data path: {DATA_PATH}")
print(f"Model output path: {MODEL_OUTPUT_PATH}")

Using project root: /Users/emmanuel/Documents/LAD
Data path: /Users/emmanuel/Documents/LAD/data/HDFS.npz
Model output path: /Users/emmanuel/Documents/LAD/ml_models/isolation_forest_model.joblib


In [20]:
# --- Load Data ---
print("--> Loading sequence data from HDFS.npz...")
with np.load(DATA_PATH, allow_pickle=True) as data:
    X_sequences = data['x_data']
    y = data['y_data']

# --- Split Data into Training and Testing Sets (BEFORE Feature Engineering) ---
print("\n--> Splitting raw sequence data into training and testing sets...")
X_seq_train, X_seq_test, y_train, y_test = train_test_split(
    X_sequences,
    y,
    test_size=0.3,
    random_state=42,
    stratify=y
 )

# Convert labels to numpy arrays for downstream use
y_train = np.array(y_train, dtype=int)
y_test = np.array(y_test, dtype=int)

print(f"Training sequences: {len(X_seq_train)}")
print(f"Test sequences: {len(X_seq_test)}")

--> Loading sequence data from HDFS.npz...

--> Splitting raw sequence data into training and testing sets...
Training sequences: 402542
Test sequences: 172519

--> Splitting raw sequence data into training and testing sets...
Training sequences: 402542
Test sequences: 172519


In [21]:
# --- Feature Engineering: Build Vocabulary on Training Data Only ---
print("\n--> Building event vocabulary from training data...")

def build_event_count_matrix(sequences, mapping):
    matrix = lil_matrix((len(sequences), len(mapping)), dtype=np.int32)
    for row_idx, seq in enumerate(sequences):
        for event in seq:
            idx = mapping.get(event)
            if idx is not None:
                matrix[row_idx, idx] += 1
    return matrix.tocsr()

train_events = [event for seq in X_seq_train for event in seq]
unique_train_events = sorted(set(train_events))
event_to_int = {event: idx for idx, event in enumerate(unique_train_events)}

print(f"Unique events in training data: {len(unique_train_events)}")

X_train = build_event_count_matrix(X_seq_train, event_to_int)
X_test = build_event_count_matrix(X_seq_test, event_to_int)

# Track unseen events in the test set (not present in training vocabulary)
unseen_events = sorted({event for seq in X_seq_test for event in seq if event not in event_to_int})
if unseen_events:
    print(f"Unseen events in test data (ignored in feature matrix): {len(unseen_events)}")
else:
    print("No unseen events in the test data.")

print("\n--- Data Summary ---")
print(f"Training matrix shape: {X_train.shape}")
print(f"Test matrix shape: {X_test.shape}")
print(f"Number of anomalies in training set: {y_train.sum()}")
print(f"Number of anomalies in test set: {y_test.sum()}")


--> Building event vocabulary from training data...
Unique events in training data: 29
No unseen events in the test data.

--- Data Summary ---
Training matrix shape: (402542, 29)
Test matrix shape: (172519, 29)
Number of anomalies in training set: 11787
Number of anomalies in test set: 5051
No unseen events in the test data.

--- Data Summary ---
Training matrix shape: (402542, 29)
Test matrix shape: (172519, 29)
Number of anomalies in training set: 11787
Number of anomalies in test set: 5051


In [22]:
# --- Hyperparameter Tuning and Model Training ---
print("\n--> Configuring model training and hyperparameter search...")

contamination_train = y_train.sum() / len(y_train)
print(f"Calculated contamination on training set: {contamination_train:.4f}")

contamination_candidates = {round(float(contamination_train), 6)}
if contamination_train < 0.05:
    contamination_candidates.add(0.05)
if contamination_train < 0.1:
    contamination_candidates.add(round(float(min(0.1, contamination_train * 1.5)), 6))
contamination_candidates = sorted(contamination_candidates)
print(f"Contamination candidates for search: {contamination_candidates}")

param_grid = {
    'n_estimators': [100, 200],
    'max_samples': [0.8, 1.0],
    'contamination': contamination_candidates,
    'max_features': [0.75, 1.0]
}

def isolation_forest_f1(y_true, y_pred):
    mapped = np.where(y_pred == -1, 1, 0)
    return f1_score(y_true, mapped)

scorer = make_scorer(isolation_forest_f1)
iso_forest = IsolationForest(random_state=42, n_jobs=-1)

grid_search = GridSearchCV(
    estimator=iso_forest,
    param_grid=param_grid,
    scoring=scorer,
    cv=3,
    n_jobs=-1,
    verbose=0
)

print("--> Starting grid search (this may take several minutes)...")
grid_search.fit(X_train, y_train)

print("\n--- Hyperparameter Search Summary ---")
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best cross-validated F1-score: {grid_search.best_score_:.4f}")

model = grid_search.best_estimator_
joblib.dump(model, MODEL_OUTPUT_PATH)
print(f"Model saved to '{MODEL_OUTPUT_PATH}'")

# --- Evaluation on Held-Out Test Set ---
print("\n--> Evaluating on the test set...")
y_test_pred = model.predict(X_test)
y_test_pred_mapped = np.where(y_test_pred == -1, 1, 0)

print("\n--- Classification Report ---")
print(classification_report(y_test, y_test_pred_mapped, target_names=['Normal', 'Anomaly']))

elapsed_minutes = (time.time() - start_time) / 60
print(f"\nTotal elapsed time: {elapsed_minutes:.2f} minutes")


--> Configuring model training and hyperparameter search...
Calculated contamination on training set: 0.0293
Contamination candidates for search: [0.029281, 0.043922, 0.05]
--> Starting grid search (this may take several minutes)...

--- Hyperparameter Search Summary ---
Best parameters: {'contamination': 0.029281, 'max_features': 0.75, 'max_samples': 1.0, 'n_estimators': 100}
Best cross-validated F1-score: 0.8005
Model saved to '/Users/emmanuel/Documents/LAD/ml_models/isolation_forest_model.joblib'

--> Evaluating on the test set...

--- Hyperparameter Search Summary ---
Best parameters: {'contamination': 0.029281, 'max_features': 0.75, 'max_samples': 1.0, 'n_estimators': 100}
Best cross-validated F1-score: 0.8005
Model saved to '/Users/emmanuel/Documents/LAD/ml_models/isolation_forest_model.joblib'

--> Evaluating on the test set...

--- Classification Report ---
              precision    recall  f1-score   support

      Normal       0.99      0.99      0.99    167468
     Anomaly